# Functional Programming

## Functions Are Just Objects

In [ ]:
a = 42
def add_one(x):
    return x + 1

In [ ]:
type(a), type(add_one)

In [ ]:
id(a), id(add_one)

In [ ]:
add_one, add_one(a)

In [ ]:
dir(add_one)

In [ ]:
add_one.__call__(42)

## Functions as Arguments

In [ ]:
from typing import Callable, Iterable, List, Any

def mapper(fn: Callable, l: Iterable[Any]) -> List[Any]:
    return [fn(x) for x in l]

In [ ]:
assert mapper(add_one, [1, 10, 100]) == [2, 11, 101]

In [ ]:
def is_even(n: int) -> bool:
    return n % 2 == 0

def filters(filter_fn: Callable, l: Iterable[Any]) -> List[Any]:
    return [x for x in l if filter_fn(x)]

In [ ]:
assert filters(is_even, [1, 2, 3, 4]) == [2, 4]

In [ ]:
import pandas as pd

In [ ]:
transaction_df = pd.DataFrame({
    'amount': [42., 100., 999.],
    'from': ['bob', 'alice', 'bob'],
    'to': ['alice', 'bob', 'alice']
})
transaction_df

In [ ]:
def select_large_transactions(transaction_df):
    return transaction_df['amount'] > 100  # Where does transaction_df refer to?

In [ ]:
transaction_df.loc[select_large_transactions]

In [ ]:
def get_commission(transaction_df):
    return transaction_df['amount'] * 0.05

In [ ]:
transaction_df.assign(commission=get_commission)

We can also provide arguments using [`functools.partial()`](https://docs.python.org/3/library/functools.html#functools.partial) from the standard library:

In [ ]:
from functools import partial

In [ ]:
def get_commission(transaction_df, commission_percent=5):
    return transaction_df['amount'] * (commission_percent / 100)

In [ ]:
transaction_df.assign(commission=partial(get_commission, commission_percent=10))

## Functions Returning Functions

In [ ]:
def add_n(n):
    def adder(x):
        return x + n
    return adder

In [ ]:
add_two = add_n(2)

In [ ]:
add_two, type(add_two)

In [ ]:
add_two(42)

In [ ]:
from functools import partial

In [ ]:
def add_n(x, n):
    return x + n

add_two = partial(add_n, n=2)
mapper(add_two, [1, 42, 100])

In [ ]:
mapper(add_one, [1, None, 100])

In [ ]:
def add_one(x):
    if x is not None:
        return x
    else:
        return None

mapper(add_one, [1, None, 100])

Why is this sub-optimal? What if there's a `add_two()` etc? Repeat all the checking for None everywhere?

In [ ]:
# Type hints omitted for brevity
def skip_None(fn):
    def fn_wrapper(n):
        if n is not None:
            return fn(n)
        else:
            return None
    
    return fn_wrapper

def add_one(x):
    return x + 1

In [ ]:
add_one_wrapped = skip_None(add_one)
mapper(add_one_wrapped, [1, None, 100])

In [ ]:
@skip_None
def add_one(x):
    return x + 1

mapper(add_one, [1, None, 100])

In [ ]:
def check_value(max_expected):
    def value_check_decorator(fn):
        def fn_wrapper(n):
            if n > max_expected:
                print(f'Unusual value {n}, expected a maximum of {max_expected}')
            return fn(n)
        return fn_wrapper
    return value_check_decorator

@check_value(max_expected=42)
def add_one(x):
    return x + 1

mapper(add_one, [1, 42, 100])

## Anonymous Functions

In [ ]:
mapper(lambda x: x + 10, [1, 42, 100])

In [ ]:
type(lambda x: x + 10)

In [ ]:
dir(lambda x: x + 10)

In [ ]:
(lambda x: x + 10)(42)

In [ ]:
# Don't do this IRL
add_ten = lambda x: x + 10
add_ten(42)

In [ ]:
from dataclasses import dataclass

In [ ]:
@dataclass
class Vector:
    values: List[float]
    
    def __getitem__(self, index: int):
        return self.values[index]
    
    def __len__(self):
        return len(self.values)
    
    def __mul__(self, scalar: float):
        return Vector([v * scalar for v in self.values])
    
    def __add__(self, other: 'Vector'):
        return Vector([self[i] + other[i] for i in range(len(self))])

__Exercise__: Add a method `pipe()` to our `Vector` class, which accepts as argument a function that transforms its input vector argument to another vector. As an example of such transformation functions, consider `rotate_right()` and `rotate_left()` given below. Test your implementation with the assertions below.

In [ ]:
def rotate_right(v: Vector) -> Vector:
    return Vector([v[1], -v[0]])

def rotate_left(v: Vector) -> Vector:
    return Vector([-v[1], v[0]])

In [ ]:
# Your solution:

In [ ]:
# %load solutions/vector_pipe.py

In [ ]:
assert Vector([1, 1]).pipe(rotate_right) == Vector([1, -1])
assert Vector([1, 1]).pipe(rotate_left).pipe(rotate_left) == Vector([-1, -1])

__Bonus Exercise__: Make sure that our `pipe()` method can accept additional arguments: create a generic `rotate()` function that accepts an optional `direction` keyword argument with possible values `'right'`, `'clockwise'`, `'left'`, `'counterclockwise'`. If an invalid or no `direction` keyword argument is provided, `rotate()` should return the original vector. See the assertions below for the expected behavior of the solution.

In [ ]:
# Your solution:

In [ ]:
# %load solutions/vector_pipe_vararg.py

In [ ]:
# bonus, have one generic rotation function
assert Vector([1, 1]).pipe(rotate) == Vector([1, 1])
assert Vector([1, 1]).pipe(rotate, direction='right').pipe(rotate, direction='counterclockwise') == Vector([1, 1])
assert Vector([1, 1]).pipe(rotate, direction='clockwise').pipe(rotate, direction='left') == Vector([1, 1])

In [ ]:
# bonus, deal with any Vector manipulation function
assert Vector([1, 1]).pipe(lambda v: Vector([v[0] * 42, v[1] * 99])) == Vector([42, 99])

In [ ]:
transaction_df

In [ ]:
def select_amount_greater_than(tx_df, amount=100):
    return tx_df.loc[lambda df: df['amount'] > amount]

In [ ]:
transaction_df.pipe(select_amount_greater_than)

In [ ]:
transaction_df.pipe(select_amount_greater_than, amount=99)

In [ ]:
transaction_df.pipe(lambda df: df.loc[df['to'].isin(['bob', 'carol'])])